<a href="https://colab.research.google.com/github/JamesBagley/James_Foundry_Tools/blob/master/Momentum_XML_worklist_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ODTC worklist generator

## Backend
# block and replace approach,
# experiments are the same, the only change is barcode & ODTC protocol
# information, which is added in via a simple string replacement
import datetime
from pathlib import PureWindowsPath
timestamp = datetime.datetime.now().timestamp()
#ODTC
ODTC = \
r'''		<batch process="Generic Spin" name="Spin1" priority="10" iterations="1" minimumDelay="0" reference="ASAP" constraint="SF">
 			<variable  type="String" name="Barcode">[Barcode]</variable>
 			<variable  type="String" name="Experiment_Name">[Experiment_Name]</variable>
			<variable  type="String" name="Plate_Type">96</variable>
		</batch>
		<batch process="Generic Seal" name="Seal1" priority="10" iterations="1" minimumDelay="0" reference="Spin1" constraint="SF">
 			<variable  type="String" name="Barcode">[Barcode]</variable>
 			<variable  type="String" name="Experiment_Name">[Experiment_Name]</variable>
			<variable  type="String" name="Plate_Type">96</variable>
		</batch>
		<batch process="Generic ODTC leave in place" name="PCR1" priority="10" iterations="1" minimumDelay="0" reference="Seal1" constraint="SF">
 			<variable  type="String" name="Barcode">[Barcode]</variable>
 			<variable  type="String" name="Experiment_Name">[Experiment_Name]</variable>
 			<variable  type="String" name="ODTC_protocol">C:\ODTC_Protocols\[Protocol_name].xml</variable>
			<variable  type="String" name="Plate_Type">96</variable>
		</batch>'''
def generate_ODTC_Block(Barcode, Protocol_name, experiment_name):
  out = ODTC.replace('[Barcode]',Barcode).replace('[Experiment_Name]', experiment_name).replace('[Protocol_name]', Protocol_name)
  return(out)
#xml = ''
#xml = xml+generate_ODTC_Block(Barcode='GF0000339', Protocol_name='96_Phire', experiment_name='')
#print(xml)
# Generate from individual modules approach
# Experiments will be very different and have to run in different orders
# but they can be grouped together into similar processes "blocks" to simplify
# their construction

def generate_xml(process, name, reference, constraint, variables):
  if constraint == False:
    constraint = 'ASAP'
    
  start = \
  f'''\t<batch process="{process}" name="{name}" priority="10" iterations="1" minimumDelay="0" reference="{reference}" constraint="{constraint}">'''
  end = '''\t</batch>'''
  var_lines = '\n'.join([f'''\t\t<variable  type={variable[0]} name={variable[1]}>{variable[2]}</variable>''' for variable in variables])
  return('\n'.join([start,var_lines,end]))

def generate_ODTC_process(name, reference, constraint, barcode, protocol, plate_type):
  return generate_xml('Generic ODTC', name, reference, constraint, [['"String"','"Barcode"', barcode],['"String"','"ODTC_protocol"', protocol],['"String"','"Plate_Type"', plate_type]])

def generate_spin_process(name, reference, constraint, barcode, plate_type):
  return generate_xml('Generic Spin', name, reference, constraint, [['"String"','"Barcode"', barcode], ['"String"','"Plate_Type"', plate_type]])

def generate_seal_process(name, reference, constraint, barcode, plate_type):
  return generate_xml('Generic Seal', name, reference, constraint, [['"String"','"Barcode"', barcode], ['"String"','"Plate_Type"', plate_type]])

def generate_mantis_process(name, reference, constraint, barcode, plate_type, dispense_list):
  return generate_xml('Generic Mantis', name, reference, constraint, [['"String"','"Barcode"', barcode], ['"String"','"Plate_Type"', plate_type], ['"String"','"Dispense List"', dispense_list]])

def generate_deseal_process(name, reference, constraint, barcode, plate_type):
  return generate_xml('Generic Deseal', name, reference, constraint, [['"String"','"Barcode"', barcode], ['"String"','"Plate_Type"', plate_type]])

def generate_ODTC_block2(barcode, protocol, plate_type, sealed_in):
  out = []
  counter = 1
  out.append(generate_spin_process('-'.join([barcode, str(counter)]), None, 'ASAP', barcode, plate_type))
  counter += 1
  if not sealed_in:
    out.append(generate_seal_process('-'.join([barcode, str(counter)]), '-'.join([barcode, str(counter-1)]), 'SF', barcode, plate_type))
    counter += 1
  out.append(generate_ODTC_process('-'.join([barcode, str(counter)]), '-'.join([barcode, str(counter-1)]), 'SF', barcode, protocol, plate_type))
  counter += 1

  return('\n'.join(out))
def generate_PCR_block(barcode, echo_transfer, mantis_dispense_list, ODTC_protocol, plate_type, sealed_in):
  out = []
  counter = 1
  out.append(generate_echo_process('-'.join([barcode, str(counter)]), None, 'ASAP', barcode, plate_type))
  counter += 1
  out.append(generate_spin_process('-'.join([barcode, str(counter)]), None, 'ASAP', barcode, plate_type))
  counter += 1
  if not sealed_in:
    out.append(generate_seal_process('-'.join([barcode, str(counter)]), '-'.join([barcode, str(counter-1)]), 'SF', barcode, plate_type))
    counter += 1
  out.append(generate_ODTC_process('-'.join([barcode, str(counter)]), '-'.join([barcode, str(counter-1)]), 'SF', barcode, protocol, plate_type))
  counter += 1
#print('\n'.join([generate_ODTC_process('Jeff', None, 'ASAP', 'GF0000842', '96_Phire', 96)]))

#print('\n\n\n---------------------------\n\n\n')

#print(generate_ODTC_block2('GF0000844', '96_Phire', 96, True))

# take string of batches and convert to work unit

def wrap_to_workunit(batches, name, append, auto_load, auto_verify_load,
                     auto_unload, auto_verify_unload):
  out = (f'''  <workunit name="{name}" append="{append}" auto_load="{auto_load}" auto_verify_load="{auto_verify_load}" auto_unload="{auto_unload}" auto_verify_unload="{auto_verify_unload}">
	{'newline'.join(batches)}
	<endworkunit>''')
  return(out.replace('<endworkunit>', '</workunit>').replace('newline','\n').replace('True', 'true').replace('False', 'false'))

def wrap_to_worklist(workunits):
 base = '<worklist>\n[workunits]\n</worklist>'
 return base.replace('[workunits]', '\n'.join(workunits))

## Interface



Barcodes = "PCR2-1, PCR2-2" #@param {type:"string"}
Barcodes = Barcodes.replace(', ', ',').split(',')
ODTC_PROTOCOL_ROOT = PureWindowsPath("C:\\ODTC_Protocols\\")
ODTC_protocol = "One-Time Use\\96_Phusion_MH part 2.xml" #@param ["96_Phire.xml", "96_YeastBoil.xml", "384_Phire.xml", "384_YeastBoil.xml"] {allow-input: true}
ODTC_protocol = ODTC_PROTOCOL_ROOT.joinpath(PureWindowsPath(ODTC_protocol))
Plate_type = "96"
Plate_type = ODTC_protocol.stem.split('_')[0]+"PCR"
#if ODTC_protocol.find(r'\\')>-1:
#  Plate_type == ODTC_protocol[ODTC_protocol.find(r'\\')+1:ODTC_protocol.find('_')-1]

Sealed = False #@param {type:"boolean"}
Initialize = False #@param {type:"boolean"}
Wait_for_human = True #@param {type:"boolean"}
protocol_name = ODTC_protocol
#ODTC_protocol ="C:\\ODTC_Protocols\\"+ODTC_protocol

workunits = []
if Initialize:
  workunits.append(
      wrap_to_workunit(
          [generate_xml('_Initialize', 'Initialization', False, False, [])],
          name='Initialize', append=False, auto_load=True, auto_verify_load=True, auto_unload=True, auto_verify_unload=True))
workunits.append(
    wrap_to_workunit(
        [generate_ODTC_block2(Barcode, ODTC_protocol, Plate_type, Sealed)
        for Barcode in Barcodes],
        name = f'ODTC workunit {timestamp}',
        append = False,
        auto_load = True,
        auto_verify_load = True,
        auto_unload = True,
        auto_verify_unload = True
        )
    )

Shutdown_processes = []
if Wait_for_human:
  Shutdown_processes.append(
      generate_xml('wait for human', 'Pause for Shutdown', False, False, [])
      )
Shutdown_processes.append(generate_xml('_Shutdown', 'Shutdown', False, False, []))
workunits.append(wrap_to_workunit(Shutdown_processes, name='Shutdown', append=False, auto_load=True, auto_verify_load=True, auto_unload=True, auto_verify_unload=True))
worklist = wrap_to_worklist(workunits)
print(worklist)
name = f'{protocol_name.stem.split(".")[0]} [{len(Barcodes)}] {timestamp}.xml'
from google.colab import files
with open(name, 'w') as f:
   f.write(worklist)
files.download(name)


In [ ]:
#@title WIP Yeast Colony PCR worklist generator process -> 384_PCR/96PCR Destination Centric { run: "auto" }
import pandas as pd
from google.colab import files
import datetime
from itertools import product
from pathlib import PureWindowsPath
from os import path
from os import system
import os
import warnings

def clear_directory(directory):
  for root, dirs, filenames in os.walk(directory, topdown=False):
    for name in filenames:
      os.remove(os.path.join(root, name))
    for name in dirs:
      os.rmdir(os.path.join(root, name))


clear_directory('.')
ECHO_ROOT = PureWindowsPath(r'\\ACCESS-1615\Labcyte\Echo\Protocols')
MANTIS_ROOT = PureWindowsPath(r'C:\Mantis-0740\Mantis-0740\Mantis\Data\User\DispenseList')

ODTC_protocol = "384_Phire.xml" #@param ["96_Phire.xml", "96_YeastBoil.xml", "384_Phire.xml", "384_YeastBoil.xml"] {allow-input: true}
Destination_plate_type = "Biorad384" #@param ["EppendorfTwintec96", "Biorad384"]
Mantis_volume = "2.5" #@param ["2.5", "50"] {allow-input: true}
Mantis_reagent = "Phire" #@param ["Phire", "Phusion"] {allow-input: true}

ODTC_PROTOCOL_ROOT = PureWindowsPath("C:\\ODTC_Protocols\\")
ODTC_protocol = ODTC_PROTOCOL_ROOT.joinpath(PureWindowsPath(ODTC_protocol))
Plate_format = ODTC_protocol.stem.split('_')[0]



if Destination_plate_type!= "Biorad384":
  warnings.warn("96 well plates not supported by momentum process")
mantis_384_template = '''[ Version: 6 ]
Bio-Rad384PCR.pd.txt
0
1
1	0	
0
Reagent1		Normal
Well	1
Reagent1-A1	Reagent1-A2	Reagent1-A3	Reagent1-A4	Reagent1-A5	Reagent1-A6	Reagent1-A7	Reagent1-A8	Reagent1-A9	Reagent1-A10	Reagent1-A11	Reagent1-A12	Reagent1-A13	Reagent1-A14	Reagent1-A15	Reagent1-A16	Reagent1-A17	Reagent1-A18	Reagent1-A19	Reagent1-A20	Reagent1-A21	Reagent1-A22	Reagent1-A23	Reagent1-A24
Reagent1-B1	Reagent1-B2	Reagent1-B3	Reagent1-B4	Reagent1-B5	Reagent1-B6	Reagent1-B7	Reagent1-B8	Reagent1-B9	Reagent1-B10	Reagent1-B11	Reagent1-B12	Reagent1-B13	Reagent1-B14	Reagent1-B15	Reagent1-B16	Reagent1-B17	Reagent1-B18	Reagent1-B19	Reagent1-B20	Reagent1-B21	Reagent1-B22	Reagent1-B23	Reagent1-B24
Reagent1-C1	Reagent1-C2	Reagent1-C3	Reagent1-C4	Reagent1-C5	Reagent1-C6	Reagent1-C7	Reagent1-C8	Reagent1-C9	Reagent1-C10	Reagent1-C11	Reagent1-C12	Reagent1-C13	Reagent1-C14	Reagent1-C15	Reagent1-C16	Reagent1-C17	Reagent1-C18	Reagent1-C19	Reagent1-C20	Reagent1-C21	Reagent1-C22	Reagent1-C23	Reagent1-C24
Reagent1-D1	Reagent1-D2	Reagent1-D3	Reagent1-D4	Reagent1-D5	Reagent1-D6	Reagent1-D7	Reagent1-D8	Reagent1-D9	Reagent1-D10	Reagent1-D11	Reagent1-D12	Reagent1-D13	Reagent1-D14	Reagent1-D15	Reagent1-D16	Reagent1-D17	Reagent1-D18	Reagent1-D19	Reagent1-D20	Reagent1-D21	Reagent1-D22	Reagent1-D23	Reagent1-D24
Reagent1-E1	Reagent1-E2	Reagent1-E3	Reagent1-E4	Reagent1-E5	Reagent1-E6	Reagent1-E7	Reagent1-E8	Reagent1-E9	Reagent1-E10	Reagent1-E11	Reagent1-E12	Reagent1-E13	Reagent1-E14	Reagent1-E15	Reagent1-E16	Reagent1-E17	Reagent1-E18	Reagent1-E19	Reagent1-E20	Reagent1-E21	Reagent1-E22	Reagent1-E23	Reagent1-E24
Reagent1-F1	Reagent1-F2	Reagent1-F3	Reagent1-F4	Reagent1-F5	Reagent1-F6	Reagent1-F7	Reagent1-F8	Reagent1-F9	Reagent1-F10	Reagent1-F11	Reagent1-F12	Reagent1-F13	Reagent1-F14	Reagent1-F15	Reagent1-F16	Reagent1-F17	Reagent1-F18	Reagent1-F19	Reagent1-F20	Reagent1-F21	Reagent1-F22	Reagent1-F23	Reagent1-F24
Reagent1-G1	Reagent1-G2	Reagent1-G3	Reagent1-G4	Reagent1-G5	Reagent1-G6	Reagent1-G7	Reagent1-G8	Reagent1-G9	Reagent1-G10	Reagent1-G11	Reagent1-G12	Reagent1-G13	Reagent1-G14	Reagent1-G15	Reagent1-G16	Reagent1-G17	Reagent1-G18	Reagent1-G19	Reagent1-G20	Reagent1-G21	Reagent1-G22	Reagent1-G23	Reagent1-G24
Reagent1-H1	Reagent1-H2	Reagent1-H3	Reagent1-H4	Reagent1-H5	Reagent1-H6	Reagent1-H7	Reagent1-H8	Reagent1-H9	Reagent1-H10	Reagent1-H11	Reagent1-H12	Reagent1-H13	Reagent1-H14	Reagent1-H15	Reagent1-H16	Reagent1-H17	Reagent1-H18	Reagent1-H19	Reagent1-H20	Reagent1-H21	Reagent1-H22	Reagent1-H23	Reagent1-H24
Reagent1-I1	Reagent1-I2	Reagent1-I3	Reagent1-I4	Reagent1-I5	Reagent1-I6	Reagent1-I7	Reagent1-I8	Reagent1-I9	Reagent1-I10	Reagent1-I11	Reagent1-I12	Reagent1-I13	Reagent1-I14	Reagent1-I15	Reagent1-I16	Reagent1-I17	Reagent1-I18	Reagent1-I19	Reagent1-I20	Reagent1-I21	Reagent1-I22	Reagent1-I23	Reagent1-I24
Reagent1-J1	Reagent1-J2	Reagent1-J3	Reagent1-J4	Reagent1-J5	Reagent1-J6	Reagent1-J7	Reagent1-J8	Reagent1-J9	Reagent1-J10	Reagent1-J11	Reagent1-J12	Reagent1-J13	Reagent1-J14	Reagent1-J15	Reagent1-J16	Reagent1-J17	Reagent1-J18	Reagent1-J19	Reagent1-J20	Reagent1-J21	Reagent1-J22	Reagent1-J23	Reagent1-J24
Reagent1-K1	Reagent1-K2	Reagent1-K3	Reagent1-K4	Reagent1-K5	Reagent1-K6	Reagent1-K7	Reagent1-K8	Reagent1-K9	Reagent1-K10	Reagent1-K11	Reagent1-K12	Reagent1-K13	Reagent1-K14	Reagent1-K15	Reagent1-K16	Reagent1-K17	Reagent1-K18	Reagent1-K19	Reagent1-K20	Reagent1-K21	Reagent1-K22	Reagent1-K23	Reagent1-K24
Reagent1-L1	Reagent1-L2	Reagent1-L3	Reagent1-L4	Reagent1-L5	Reagent1-L6	Reagent1-L7	Reagent1-L8	Reagent1-L9	Reagent1-L10	Reagent1-L11	Reagent1-L12	Reagent1-L13	Reagent1-L14	Reagent1-L15	Reagent1-L16	Reagent1-L17	Reagent1-L18	Reagent1-L19	Reagent1-L20	Reagent1-L21	Reagent1-L22	Reagent1-L23	Reagent1-L24
Reagent1-M1	Reagent1-M2	Reagent1-M3	Reagent1-M4	Reagent1-M5	Reagent1-M6	Reagent1-M7	Reagent1-M8	Reagent1-M9	Reagent1-M10	Reagent1-M11	Reagent1-M12	Reagent1-M13	Reagent1-M14	Reagent1-M15	Reagent1-M16	Reagent1-M17	Reagent1-M18	Reagent1-M19	Reagent1-M20	Reagent1-M21	Reagent1-M22	Reagent1-M23	Reagent1-M24
Reagent1-N1	Reagent1-N2	Reagent1-N3	Reagent1-N4	Reagent1-N5	Reagent1-N6	Reagent1-N7	Reagent1-N8	Reagent1-N9	Reagent1-N10	Reagent1-N11	Reagent1-N12	Reagent1-N13	Reagent1-N14	Reagent1-N15	Reagent1-N16	Reagent1-N17	Reagent1-N18	Reagent1-N19	Reagent1-N20	Reagent1-N21	Reagent1-N22	Reagent1-N23	Reagent1-N24
Reagent1-O1	Reagent1-O2	Reagent1-O3	Reagent1-O4	Reagent1-O5	Reagent1-O6	Reagent1-O7	Reagent1-O8	Reagent1-O9	Reagent1-O10	Reagent1-O11	Reagent1-O12	Reagent1-O13	Reagent1-O14	Reagent1-O15	Reagent1-O16	Reagent1-O17	Reagent1-O18	Reagent1-O19	Reagent1-O20	Reagent1-O21	Reagent1-O22	Reagent1-O23	Reagent1-O24
Reagent1-P1	Reagent1-P2	Reagent1-P3	Reagent1-P4	Reagent1-P5	Reagent1-P6	Reagent1-P7	Reagent1-P8	Reagent1-P9	Reagent1-P10	Reagent1-P11	Reagent1-P12	Reagent1-P13	Reagent1-P14	Reagent1-P15	Reagent1-P16	Reagent1-P17	Reagent1-P18	Reagent1-P19	Reagent1-P20	Reagent1-P21	Reagent1-P22	Reagent1-P23	Reagent1-P24


'''
mantis_96_template = '''[ Version: 6 ]
Eppendorf twin.tec PCR_96.pd.txt
0
1
1	0	
0
Reagent1		Normal
Well	1
Reagent1-A1	Reagent1-A2	Reagent1-A3	Reagent1-A4	Reagent1-A5	Reagent1-A6	Reagent1-A7	Reagent1-A8	Reagent1-A9	Reagent1-A10	Reagent1-A11	Reagent1-A12
Reagent1-B1	Reagent1-B2	Reagent1-B3	Reagent1-B4	Reagent1-B5	Reagent1-B6	Reagent1-B7	Reagent1-B8	Reagent1-B9	Reagent1-B10	Reagent1-B11	Reagent1-B12
Reagent1-C1	Reagent1-C2	Reagent1-C3	Reagent1-C4	Reagent1-C5	Reagent1-C6	Reagent1-C7	Reagent1-C8	Reagent1-C9	Reagent1-C10	Reagent1-C11	Reagent1-C12
Reagent1-D1	Reagent1-D2	Reagent1-D3	Reagent1-D4	Reagent1-D5	Reagent1-D6	Reagent1-D7	Reagent1-D8	Reagent1-D9	Reagent1-D10	Reagent1-D11	Reagent1-D12
Reagent1-E1	Reagent1-E2	Reagent1-E3	Reagent1-E4	Reagent1-E5	Reagent1-E6	Reagent1-E7	Reagent1-E8	Reagent1-E9	Reagent1-E10	Reagent1-E11	Reagent1-E12
Reagent1-F1	Reagent1-F2	Reagent1-F3	Reagent1-F4	Reagent1-F5	Reagent1-F6	Reagent1-F7	Reagent1-F8	Reagent1-F9	Reagent1-F10	Reagent1-F11	Reagent1-F12
Reagent1-G1	Reagent1-G2	Reagent1-G3	Reagent1-G4	Reagent1-G5	Reagent1-G6	Reagent1-G7	Reagent1-G8	Reagent1-G9	Reagent1-G10	Reagent1-G11	Reagent1-G12
Reagent1-H1	Reagent1-H2	Reagent1-H3	Reagent1-H4	Reagent1-H5	Reagent1-H6	Reagent1-H7	Reagent1-H8	Reagent1-H9	Reagent1-H10	Reagent1-H11	Reagent1-H12

'''

Mantis_template_files = {
    "EppendorfTwintec96":mantis_96_template,
    "Biorad384":mantis_384_template
}
##
## Whole experiment structure
##
date = datetime.datetime.utcnow().strftime('%Y-%m-%dT%H%M%SZ')
experiment_name = "Yeast cPCR" + date
path = './'+date
upload = next(iter(files.upload()))
Echo_file = pd.read_csv(upload)
os.remove(upload)


Echo_file['Source Plate Barcode'] = Echo_file['Source Plate Name']
Destination_barcodes = Echo_file['Destination Plate Barcode'].drop_duplicates()

##
## Mantis back end code
##

def get_plate_type(template):
  if template.find("384")>-1:
    return 384
  elif template.find("96")>-1:
    return 96
  else:
    return 'unknown'

def generate_mantis_dispense_list(reagent, volume, input, template):
  template = template.replace('Reagent1', reagent)
  Rows_384 = 'ABCDEFGHIJKLMNOP'
  Cols_384 = '1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24'.split(',')
  wells_384 = list(map(lambda x: x[0]+x[1], product(Rows_384, Cols_384)))
  for well in wells_384[::-1]:
    try:
        template = template.replace(f'{reagent}-{well}',
                                      str(input.loc[well]))
    except(KeyError):
        template = template.replace(f'{reagent}-{well}',
                                      str(0))
  return template.replace('\n','\r\n')

def parse_for_mantis(rows):
  index = rows['Destination Well']
  out = pd.Series(index=index, data=[volume_uL]*len(index), name=reagent)
  return out
  
##
## Generating dispense list files for mantis
##

reagent = Mantis_reagent
volume_uL = Mantis_volume
mantis_template = Mantis_template_files[Destination_plate_type]

mantis_files = Echo_file.iloc(axis=1)[5:].drop_duplicates(
    ['Destination Plate Barcode',
     'Destination Plate Name',
     'Destination Well']).\
     groupby(by='Destination Plate Barcode').\
     apply(parse_for_mantis).apply(
      lambda destination_transfers: generate_mantis_dispense_list(
          reagent=Mantis_reagent,
          volume=Mantis_volume,
          input=destination_transfers,
          template=mantis_template),
        axis=1)

###
### Generating XMLs for Momentum
###

ODTC_PROTOCOL_ROOT = PureWindowsPath("C:\\ODTC_Protocols\\")

ODTC_protocol = ODTC_PROTOCOL_ROOT.joinpath(PureWindowsPath(ODTC_protocol))
Plate_type = ODTC_protocol.stem.split('_')[0]+"PCR"

batch_file = ''
filenames={}
for barcode in Destination_barcodes.values:
  plate={}
  mantis = mantis_files[barcode]
  mantis_filename = f'{barcode} yeast cPCR mantis {date}.dl.txt'
  echo = Echo_file[Echo_file['Destination Plate Barcode']==barcode]
  echo_filename = f'{barcode} yeast cPCR echo {date}.csv'
  with open(mantis_filename, 'w') as f:
    f.write(mantis)
  plate['mantis'] = MANTIS_ROOT.joinpath(mantis_filename)
  batch_line = f'xcopy "{mantis_filename}" "{MANTIS_ROOT}"\n'
  batch_file = batch_file+batch_line
  echo.to_csv(echo_filename, index=False)
  plate['echo'] = ECHO_ROOT.joinpath(echo_filename)
  batch_line = f'xcopy "{echo_filename}" "{ECHO_ROOT}"'
  batch_file = batch_file+batch_line
  plate['ODTC'] = str(ODTC_protocol)
  
  filenames[barcode] = plate

with open(f'yeast cPCR {date}.bat', 'w') as f:
  f.write(batch_file)

dataminer = pd.DataFrame(filenames).T
dataminer.reset_index().rename(columns={'index':'Exp_plate'}).set_index('Exp_plate').to_csv('dataminer.csv')
timestamp = datetime.datetime.now().timestamp()

def generate_xml(process, name, reference, constraint, variables):
  if constraint == False:
    constraint = 'ASAP'
    
  start = \
  f'''\t<batch process="{process}" name="{name}" priority="10" iterations="1" minimumDelay="0" reference="{reference}" constraint="{constraint}">'''
  end = '''\t</batch>'''
  var_lines = '\n'.join([f'''\t\t<variable  type={variable[0]} name={variable[1]}>{variable[2]}</variable>''' for variable in variables])
  return('\n'.join([start,var_lines,end]))

def wrap_to_workunit(batches, name, append, auto_load, auto_verify_load,
                     auto_unload, auto_verify_unload):
  out = (f'''  <workunit name="{name}" append="{append}" auto_load="{auto_load}" auto_verify_load="{auto_verify_load}" auto_unload="{auto_unload}" auto_verify_unload="{auto_verify_unload}">
	{'newline'.join(batches)}
	<endworkunit>''')
  return(out.replace('<endworkunit>', '</workunit>').replace('newline','\n').replace('True', 'true').replace('False', 'false'))

def wrap_to_worklist(workunits):
 base = '<worklist>\n[workunits]\n</worklist>'
 return base.replace('[workunits]', '\n'.join(workunits))

Initialize = True
Wait_for_human = True


DOWNLOAD_ROOT = PureWindowsPath(r'C:\Users\Thermo\Downloads\File Lander')

PCR = \
r'''
      <batch process="{Plate_format}_PCR Destination Centric" name="PCR_$DestinationBarcode" priority="10" iterations="1" minimumDelay="0">
			<variable  type="String" name="Barcode">$DestinationBarcode</variable>
			<variable  type="String" name="DataMiner">$DataminerPath</variable>
			</batch>
      '''.format(Plate_format=Plate_format)
PCR_XML = ''.join([PCR.replace('$DestinationBarcode', barcode).replace('$DataminerPath', str(DOWNLOAD_ROOT.joinpath(f'{date}').joinpath('dataminer.csv'))) for barcode in Destination_barcodes.values])
workunits = []
if Initialize:
  workunits.append(
      wrap_to_workunit(
          [generate_xml('_Initialize', 'Initialization', False, False, [])],
          name='Initialize', append=False, auto_load=True, auto_verify_load=True, auto_unload=True, auto_verify_unload=True))
workunits.append(
    wrap_to_workunit(
        [PCR_XML],
        name = f'ODTC workunit {timestamp}',
        append = False,
        auto_load = True,
        auto_verify_load = True,
        auto_unload = True,
        auto_verify_unload = True
        )
    )

Shutdown_processes = []
if Wait_for_human:
  Shutdown_processes.append(
      generate_xml('wait for human', 'Pause for Shutdown', False, False, [])
      )
Shutdown_processes.append(generate_xml('_Shutdown', 'Shutdown', False, False, []))
workunits.append(wrap_to_workunit(Shutdown_processes, name='Shutdown', append=False, auto_load=True, auto_verify_load=True, auto_unload=True, auto_verify_unload=True))
worklist = wrap_to_worklist(workunits)

with open(f'yeast cPCR {date}.xml', 'w') as f:
  f.write(worklist)

#print(filenames)
import shutil
shutil.copytree('/content', f'/{date}')

os.system("zip -r {} {}".format(f'{date}.zip', f'/{date}'))
files.download(f'{date}.zip')
#os.kill(os.getpid(), 9)
#!zip -r path



In [ ]:
#@title Defuncy Yeast Colony PCR worklist generator process -> 384_PCR Destination Centric
import pandas as pd
from google.colab import files
import datetime
from itertools import product
from pathlib import PureWindowsPath
from os import path
from os import system
import os
import warnings

for root, dirs, filenames in os.walk(".", topdown=False):
   for name in filenames:
      os.remove(os.path.join(root, name))
   for name in dirs:
      os.rmdir(os.path.join(root, name))

ECHO_ROOT = PureWindowsPath(r'\\ACCESS-1615\Labcyte\Echo\Protocols')
MANTIS_ROOT = PureWindowsPath(r'C:\Mantis-0740\Mantis-0740\Mantis\Data\User\DispenseList')

ODTC_protocol = "384_Phire.xml" #@param ["96_Phire.xml", "96_YeastBoil.xml", "384_Phire.xml", "384_YeastBoil.xml"] {allow-input: true}
Destination_plate_type = "Biorad384" #@param ["EppendorfTwintec96", "Biorad384"]


if Destination_plate_type!= "Biorad384":
  warnings.warn("96 well plates not supported by momentum process")
mantis_384_template = '''[ Version: 6 ]
Bio-Rad384PCR.pd.txt
0
1
1	0	
0
Reagent1		Normal
Well	1
Reagent1-A1	Reagent1-A2	Reagent1-A3	Reagent1-A4	Reagent1-A5	Reagent1-A6	Reagent1-A7	Reagent1-A8	Reagent1-A9	Reagent1-A10	Reagent1-A11	Reagent1-A12	Reagent1-A13	Reagent1-A14	Reagent1-A15	Reagent1-A16	Reagent1-A17	Reagent1-A18	Reagent1-A19	Reagent1-A20	Reagent1-A21	Reagent1-A22	Reagent1-A23	Reagent1-A24
Reagent1-B1	Reagent1-B2	Reagent1-B3	Reagent1-B4	Reagent1-B5	Reagent1-B6	Reagent1-B7	Reagent1-B8	Reagent1-B9	Reagent1-B10	Reagent1-B11	Reagent1-B12	Reagent1-B13	Reagent1-B14	Reagent1-B15	Reagent1-B16	Reagent1-B17	Reagent1-B18	Reagent1-B19	Reagent1-B20	Reagent1-B21	Reagent1-B22	Reagent1-B23	Reagent1-B24
Reagent1-C1	Reagent1-C2	Reagent1-C3	Reagent1-C4	Reagent1-C5	Reagent1-C6	Reagent1-C7	Reagent1-C8	Reagent1-C9	Reagent1-C10	Reagent1-C11	Reagent1-C12	Reagent1-C13	Reagent1-C14	Reagent1-C15	Reagent1-C16	Reagent1-C17	Reagent1-C18	Reagent1-C19	Reagent1-C20	Reagent1-C21	Reagent1-C22	Reagent1-C23	Reagent1-C24
Reagent1-D1	Reagent1-D2	Reagent1-D3	Reagent1-D4	Reagent1-D5	Reagent1-D6	Reagent1-D7	Reagent1-D8	Reagent1-D9	Reagent1-D10	Reagent1-D11	Reagent1-D12	Reagent1-D13	Reagent1-D14	Reagent1-D15	Reagent1-D16	Reagent1-D17	Reagent1-D18	Reagent1-D19	Reagent1-D20	Reagent1-D21	Reagent1-D22	Reagent1-D23	Reagent1-D24
Reagent1-E1	Reagent1-E2	Reagent1-E3	Reagent1-E4	Reagent1-E5	Reagent1-E6	Reagent1-E7	Reagent1-E8	Reagent1-E9	Reagent1-E10	Reagent1-E11	Reagent1-E12	Reagent1-E13	Reagent1-E14	Reagent1-E15	Reagent1-E16	Reagent1-E17	Reagent1-E18	Reagent1-E19	Reagent1-E20	Reagent1-E21	Reagent1-E22	Reagent1-E23	Reagent1-E24
Reagent1-F1	Reagent1-F2	Reagent1-F3	Reagent1-F4	Reagent1-F5	Reagent1-F6	Reagent1-F7	Reagent1-F8	Reagent1-F9	Reagent1-F10	Reagent1-F11	Reagent1-F12	Reagent1-F13	Reagent1-F14	Reagent1-F15	Reagent1-F16	Reagent1-F17	Reagent1-F18	Reagent1-F19	Reagent1-F20	Reagent1-F21	Reagent1-F22	Reagent1-F23	Reagent1-F24
Reagent1-G1	Reagent1-G2	Reagent1-G3	Reagent1-G4	Reagent1-G5	Reagent1-G6	Reagent1-G7	Reagent1-G8	Reagent1-G9	Reagent1-G10	Reagent1-G11	Reagent1-G12	Reagent1-G13	Reagent1-G14	Reagent1-G15	Reagent1-G16	Reagent1-G17	Reagent1-G18	Reagent1-G19	Reagent1-G20	Reagent1-G21	Reagent1-G22	Reagent1-G23	Reagent1-G24
Reagent1-H1	Reagent1-H2	Reagent1-H3	Reagent1-H4	Reagent1-H5	Reagent1-H6	Reagent1-H7	Reagent1-H8	Reagent1-H9	Reagent1-H10	Reagent1-H11	Reagent1-H12	Reagent1-H13	Reagent1-H14	Reagent1-H15	Reagent1-H16	Reagent1-H17	Reagent1-H18	Reagent1-H19	Reagent1-H20	Reagent1-H21	Reagent1-H22	Reagent1-H23	Reagent1-H24
Reagent1-I1	Reagent1-I2	Reagent1-I3	Reagent1-I4	Reagent1-I5	Reagent1-I6	Reagent1-I7	Reagent1-I8	Reagent1-I9	Reagent1-I10	Reagent1-I11	Reagent1-I12	Reagent1-I13	Reagent1-I14	Reagent1-I15	Reagent1-I16	Reagent1-I17	Reagent1-I18	Reagent1-I19	Reagent1-I20	Reagent1-I21	Reagent1-I22	Reagent1-I23	Reagent1-I24
Reagent1-J1	Reagent1-J2	Reagent1-J3	Reagent1-J4	Reagent1-J5	Reagent1-J6	Reagent1-J7	Reagent1-J8	Reagent1-J9	Reagent1-J10	Reagent1-J11	Reagent1-J12	Reagent1-J13	Reagent1-J14	Reagent1-J15	Reagent1-J16	Reagent1-J17	Reagent1-J18	Reagent1-J19	Reagent1-J20	Reagent1-J21	Reagent1-J22	Reagent1-J23	Reagent1-J24
Reagent1-K1	Reagent1-K2	Reagent1-K3	Reagent1-K4	Reagent1-K5	Reagent1-K6	Reagent1-K7	Reagent1-K8	Reagent1-K9	Reagent1-K10	Reagent1-K11	Reagent1-K12	Reagent1-K13	Reagent1-K14	Reagent1-K15	Reagent1-K16	Reagent1-K17	Reagent1-K18	Reagent1-K19	Reagent1-K20	Reagent1-K21	Reagent1-K22	Reagent1-K23	Reagent1-K24
Reagent1-L1	Reagent1-L2	Reagent1-L3	Reagent1-L4	Reagent1-L5	Reagent1-L6	Reagent1-L7	Reagent1-L8	Reagent1-L9	Reagent1-L10	Reagent1-L11	Reagent1-L12	Reagent1-L13	Reagent1-L14	Reagent1-L15	Reagent1-L16	Reagent1-L17	Reagent1-L18	Reagent1-L19	Reagent1-L20	Reagent1-L21	Reagent1-L22	Reagent1-L23	Reagent1-L24
Reagent1-M1	Reagent1-M2	Reagent1-M3	Reagent1-M4	Reagent1-M5	Reagent1-M6	Reagent1-M7	Reagent1-M8	Reagent1-M9	Reagent1-M10	Reagent1-M11	Reagent1-M12	Reagent1-M13	Reagent1-M14	Reagent1-M15	Reagent1-M16	Reagent1-M17	Reagent1-M18	Reagent1-M19	Reagent1-M20	Reagent1-M21	Reagent1-M22	Reagent1-M23	Reagent1-M24
Reagent1-N1	Reagent1-N2	Reagent1-N3	Reagent1-N4	Reagent1-N5	Reagent1-N6	Reagent1-N7	Reagent1-N8	Reagent1-N9	Reagent1-N10	Reagent1-N11	Reagent1-N12	Reagent1-N13	Reagent1-N14	Reagent1-N15	Reagent1-N16	Reagent1-N17	Reagent1-N18	Reagent1-N19	Reagent1-N20	Reagent1-N21	Reagent1-N22	Reagent1-N23	Reagent1-N24
Reagent1-O1	Reagent1-O2	Reagent1-O3	Reagent1-O4	Reagent1-O5	Reagent1-O6	Reagent1-O7	Reagent1-O8	Reagent1-O9	Reagent1-O10	Reagent1-O11	Reagent1-O12	Reagent1-O13	Reagent1-O14	Reagent1-O15	Reagent1-O16	Reagent1-O17	Reagent1-O18	Reagent1-O19	Reagent1-O20	Reagent1-O21	Reagent1-O22	Reagent1-O23	Reagent1-O24
Reagent1-P1	Reagent1-P2	Reagent1-P3	Reagent1-P4	Reagent1-P5	Reagent1-P6	Reagent1-P7	Reagent1-P8	Reagent1-P9	Reagent1-P10	Reagent1-P11	Reagent1-P12	Reagent1-P13	Reagent1-P14	Reagent1-P15	Reagent1-P16	Reagent1-P17	Reagent1-P18	Reagent1-P19	Reagent1-P20	Reagent1-P21	Reagent1-P22	Reagent1-P23	Reagent1-P24


'''
mantis_96_template = '''[ Version: 6 ]
Eppendorf twin.tec PCR_96.pd.txt
0
1
1	0	
0
Reagent1		Normal
Well	1
Reagent1-A1	Reagent1-A2	Reagent1-A3	Reagent1-A4	Reagent1-A5	Reagent1-A6	Reagent1-A7	Reagent1-A8	Reagent1-A9	Reagent1-A10	Reagent1-A11	Reagent1-A12
Reagent1-B1	Reagent1-B2	Reagent1-B3	Reagent1-B4	Reagent1-B5	Reagent1-B6	Reagent1-B7	Reagent1-B8	Reagent1-B9	Reagent1-B10	Reagent1-B11	Reagent1-B12
Reagent1-C1	Reagent1-C2	Reagent1-C3	Reagent1-C4	Reagent1-C5	Reagent1-C6	Reagent1-C7	Reagent1-C8	Reagent1-C9	Reagent1-C10	Reagent1-C11	Reagent1-C12
Reagent1-D1	Reagent1-D2	Reagent1-D3	Reagent1-D4	Reagent1-D5	Reagent1-D6	Reagent1-D7	Reagent1-D8	Reagent1-D9	Reagent1-D10	Reagent1-D11	Reagent1-D12
Reagent1-E1	Reagent1-E2	Reagent1-E3	Reagent1-E4	Reagent1-E5	Reagent1-E6	Reagent1-E7	Reagent1-E8	Reagent1-E9	Reagent1-E10	Reagent1-E11	Reagent1-E12
Reagent1-F1	Reagent1-F2	Reagent1-F3	Reagent1-F4	Reagent1-F5	Reagent1-F6	Reagent1-F7	Reagent1-F8	Reagent1-F9	Reagent1-F10	Reagent1-F11	Reagent1-F12
Reagent1-G1	Reagent1-G2	Reagent1-G3	Reagent1-G4	Reagent1-G5	Reagent1-G6	Reagent1-G7	Reagent1-G8	Reagent1-G9	Reagent1-G10	Reagent1-G11	Reagent1-G12
Reagent1-H1	Reagent1-H2	Reagent1-H3	Reagent1-H4	Reagent1-H5	Reagent1-H6	Reagent1-H7	Reagent1-H8	Reagent1-H9	Reagent1-H10	Reagent1-H11	Reagent1-H12

'''

Mantis_template_files = {
    "EppendorfTwintec96":mantis_96_template,
    "Biorad384":mantis_384_template
}
##
## Whole experiment structure
##

experiment_name = "Yeast cPCR" + datetime.datetime.utcnow().strftime(
                                                        '%Y-%m-%dT%H%M%SZ')
path = './'+datetime.datetime.utcnow().strftime(
                                                        '%Y-%m-%dT%H%M%SZ')
upload = next(iter(files.upload()))
Echo_file = pd.read_csv(upload)
os.remove(upload)
Echo_file['Source Plate Barcode'] = Echo_file['Source Plate Name']
Destination_barcodes = Echo_file['Destination Plate Barcode'].drop_duplicates()

##
## Mantis back end code
##

Rows_96 = 'ABCDEFGH'
Cols_96 = '1,2,3,4,5,6,7,8,9,10,11,12'.split(',')
wells_96 = list(map(lambda x: x[0]+x[1], product(Rows_96, Cols_96)))

Rows_384 = 'ABCDEFGHIJKLMNOP'
Cols_384 = '1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24'.split(',')
wells_384 = list(map(lambda x: x[0]+x[1], product(Rows_384, Cols_384)))

def get_plate_type(template):
  if template.find("384")>-1:
    return 384
  elif template.find("96")>-1:
    return 96
  else:
    return 'unknown'

def generate_mantis_dispense_list(reagent, input):
  mantis_file = Mantis_template_files[Destination_plate_type]#''.join(open('/content/Biorad384PCR Mantis template.dl.txt').readlines())
  mantis_file = mantis_file.replace('Reagent1', reagent)
  templates = {96:wells_96,
               384:wells_384}
  template = templates[get_plate_type(mantis_file)]
  for well in template[::-1]:
    try:
        mantis_file = mantis_file.replace(f'{reagent}-{well}',
                                      str(input.loc[well]))
    except(KeyError):
        mantis_file = mantis_file.replace(f'{reagent}-{well}',
                                      str(0))
  return mantis_file.replace('\n','\r\n')

##
## Generating dispense list files for mantis
##

reagent = "Phire"
volume_uL = 2.5 

input = Echo_file.iloc(axis=1)[5:]

def parse_for_mantis(rows):
  index = rows['Destination Well']
  out = pd.Series(index=index, data=[volume_uL]*len(index), name=reagent)
  return out

mantis_files = input.drop_duplicates(
    ['Destination Plate Barcode',
     'Destination Plate Name',
     'Destination Well']).\
     groupby(by='Destination Plate Barcode').\
     apply(parse_for_mantis).apply(
    lambda x: generate_mantis_dispense_list(reagent, x), axis=1)

date = datetime.datetime.utcnow().strftime('%Y-%m-%dT%H%M%SZ')


###
### Generating XMLs for Momentum
###

ODTC_PROTOCOL_ROOT = PureWindowsPath("C:\\ODTC_Protocols\\")

ODTC_protocol = ODTC_PROTOCOL_ROOT.joinpath(PureWindowsPath(ODTC_protocol))
Plate_type = ODTC_protocol.stem.split('_')[0]+"PCR"

batch_file = ''
filenames={}
for barcode in Destination_barcodes.values:
  plate={}
  mantis = mantis_files[barcode]
  mantis_filename = f'{barcode} yeast cPCR mantis {date}.dl.txt'
  echo = Echo_file[Echo_file['Destination Plate Barcode']==barcode]
  echo_filename = f'{barcode} yeast cPCR echo {date}.csv'
  with open(mantis_filename, 'w') as f:
    f.write(mantis)
  plate['mantis'] = MANTIS_ROOT.joinpath(mantis_filename)
  batch_line = f'xcopy "{mantis_filename}" "{MANTIS_ROOT}"\n'
  batch_file = batch_file+batch_line
  echo.to_csv(echo_filename)
  plate['echo'] = ECHO_ROOT.joinpath(echo_filename)
  batch_line = f'xcopy "{echo_filename}" "{ECHO_ROOT}"'
  batch_file = batch_file+batch_line
  plate['ODTC'] = str(ODTC_protocol)
  
  filenames[barcode] = plate

with open(f'yeast cPCR {date}.bat', 'w') as f:
  f.write(batch_file)

dataminer = pd.DataFrame(filenames).T
dataminer.reset_index().rename(columns={'index':'Exp_plate'}).set_index('Exp_plate').to_csv('dataminer.csv')
timestamp = datetime.datetime.now().timestamp()

def generate_xml(process, name, reference, constraint, variables):
  if constraint == False:
    constraint = 'ASAP'
    
  start = \
  f'''\t<batch process="{process}" name="{name}" priority="10" iterations="1" minimumDelay="0" reference="{reference}" constraint="{constraint}">'''
  end = '''\t</batch>'''
  var_lines = '\n'.join([f'''\t\t<variable  type={variable[0]} name={variable[1]}>{variable[2]}</variable>''' for variable in variables])
  return('\n'.join([start,var_lines,end]))

def wrap_to_workunit(batches, name, append, auto_load, auto_verify_load,
                     auto_unload, auto_verify_unload):
  out = (f'''  <workunit name="{name}" append="{append}" auto_load="{auto_load}" auto_verify_load="{auto_verify_load}" auto_unload="{auto_unload}" auto_verify_unload="{auto_verify_unload}">
	{'newline'.join(batches)}
	<endworkunit>''')
  return(out.replace('<endworkunit>', '</workunit>').replace('newline','\n').replace('True', 'true').replace('False', 'false'))

def wrap_to_worklist(workunits):
 base = '<worklist>\n[workunits]\n</worklist>'
 return base.replace('[workunits]', '\n'.join(workunits))

Initialize = True
Wait_for_human = True


DOWNLOAD_ROOT = PureWindowsPath(r'C:\Users\Thermo\Downloads\File Lander')

PCR = \
r'''
      <batch process="384_PCR Destination Centric" name="PCR_$DestinationBarcode" priority="10" iterations="1" minimumDelay="0">
			<variable  type="String" name="Barcode">$DestinationBarcode</variable>
			<variable  type="String" name="DataMiner">$DataminerPath</variable>
			</batch>
      '''
PCR_XML = ''.join([PCR.replace('$DestinationBarcode', barcode).replace('$DataminerPath', str(DOWNLOAD_ROOT.joinpath(f'{date}').joinpath('dataminer.csv'))) for barcode in Destination_barcodes.values])
workunits = []
if Initialize:
  workunits.append(
      wrap_to_workunit(
          [generate_xml('_Initialize', 'Initialization', False, False, [])],
          name='Initialize', append=False, auto_load=True, auto_verify_load=True, auto_unload=True, auto_verify_unload=True))
workunits.append(
    wrap_to_workunit(
        [PCR_XML],
        name = f'ODTC workunit {timestamp}',
        append = False,
        auto_load = True,
        auto_verify_load = True,
        auto_unload = True,
        auto_verify_unload = True
        )
    )

Shutdown_processes = []
if Wait_for_human:
  Shutdown_processes.append(
      generate_xml('wait for human', 'Pause for Shutdown', False, False, [])
      )
Shutdown_processes.append(generate_xml('_Shutdown', 'Shutdown', False, False, []))
workunits.append(wrap_to_workunit(Shutdown_processes, name='Shutdown', append=False, auto_load=True, auto_verify_load=True, auto_unload=True, auto_verify_unload=True))
worklist = wrap_to_worklist(workunits)

with open(f'yeast cPCR {date}.xml', 'w') as f:
  f.write(worklist)

#print(filenames)
import shutil
shutil.copytree('/content', f'/{date}')

os.system("zip -r {} {}".format(f'{date}.zip', f'/{date}'))
files.download(f'{date}.zip')
#os.kill(os.getpid(), 9)
#!zip -r path



In [ ]:
#@title Defunct Yeast Colony PCR worklist generator


##
## Mantis back end code
##
import pandas as pd
from google.colab import files
import datetime
from itertools import product
Rows_96 = 'ABCDEFGH'
Cols_96 = '1,2,3,4,5,6,7,8,9,10,11,12'.split(',')
wells_96 = list(map(lambda x: x[0]+x[1], product(Rows_96, Cols_96)))

Rows_384 = 'ABCDEFGHIJKMNOP'
Cols_384 = '1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24'.split(',')
wells_384 = list(map(lambda x: x[0]+x[1], product(Rows_384, Cols_384)))

def get_plate_type(template):
  if template.find("384")>-1:
    return 384
  elif template.find("96")>-1:
    return 96
  else:
    return 'unknown'

def generate_mantis_dispense_list(reagent, input):
  mantis_file = ''.join(open('Mantis template.dl.txt').readlines())
  mantis_file = mantis_file.replace('Reagent1', reagent)
  templates = {96:wells_96,
               384:wells_384}
  template = templates[get_plate_type(mantis_file)]
  for well in template[::-1]:
    try:
        mantis_file = mantis_file.replace(f'{reagent}-{well}',
                                      str(input.loc[well]))
    except(KeyError):
        mantis_file = mantis_file.replace(f'{reagent}-{well}',
                                      str(0))
  return mantis_file

##
## Generating dispense list files for mantis
##

reagent = "Phire"
volume_uL = 2.5 

input = next(iter(files.upload()))
input = pd.read_csv(input)
input = input.iloc(axis=1)[5:]

Barcodes = input['Destination Plate Barcode'].drop_duplicates()


def parse_for_mantis(rows):
  name = rows['Destination Plate Barcode'][0]
  index = rows['Destination Well']
  out = pd.Series(index=index, data=[volume_uL]*len(index), name=reagent)
  return out

mantis_files = input.drop_duplicates(
    ['Destination Plate Barcode',
     'Destination Plate Name',
     'Destination Well'])\
.groupby(by='Destination Plate Barcode').apply(parse_for_mantis).apply(
    lambda x: generate_mantis_dispense_list(reagent, x), axis=1)

date = datetime.datetime.utcnow().strftime('%Y-%m-%dT%H%M%SZ')
filenames = {}
for plate, file in mantis_files.iteritems():
  filenames[plate:f'Mantis {plate} {date}.dl.txt']
  with open(filenames[plate], 'w') as f:
    f.write(file)

###
### Generating XMLs for Momentum
###

ODTC_PROTOCOL_ROOT = PureWindowsPath("C:\\ODTC_Protocols\\")
ODTC_protocol = "One-Time Use\\96_Phusion_MH part 2.xml" #@param ["96_Phire.xml", "96_YeastBoil.xml", "384_Phire.xml", "384_YeastBoil.xml"] {allow-input: true}
ODTC_protocol = ODTC_PROTOCOL_ROOT.joinpath(PureWindowsPath(ODTC_protocol))
Plate_type = "96"
Plate_type = ODTC_protocol.stem.split('_')[0]+"PCR"


Initialize = False #@param {type:"boolean"}
Wait_for_human = True #@param {type:"boolean"}
protocol_name = ODTC_protocol

workunits = []
if Initialize:
  workunits.append(
      wrap_to_workunit(
          [generate_xml('_Initialize', 'Initialization', False, False, [])],
          name='Initialize', append=False, auto_load=True, auto_verify_load=True, auto_unload=True, auto_verify_unload=True))
  
workunits.append(wrap_to_workunit([generate_ODTC_block2(Barcode, ODTC_protocol, Plate_type, Sealed) for Barcode in Barcodes], f'ODTC workunit {timestamp}', False, True, True, True, True))
Shutdown_processes = []
if Wait_for_human:
  Shutdown_processes.append(generate_xml('wait for human', 'Pause for Shutdown', False, False, []))
Shutdown_processes.append(generate_xml('_Shutdown', 'Shutdown', False, False, []))
workunits.append(wrap_to_workunit(Shutdown_processes, name='Shutdown', append=False, auto_load=True, auto_verify_load=True, auto_unload=True, auto_verify_unload=True))
worklist = wrap_to_worklist(workunits)
print(worklist)
name = f'{protocol_name.stem.split(".")[0]} [{len(Barcodes)}] {timestamp}.xml'
from google.colab import files
with open(name, 'w') as f:
   f.write(worklist)
